In [1]:
import dgl
from dgl.data.utils import load_graphs
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from model.model import HTGNN, NodePredictor
from utils.pytorchtools import EarlyStopping
from sklearn import metrics

dgl.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [2]:
# device = torch.device('cuda:1')
device = torch.device('cpu')

In [3]:

train_feats, _ = load_graphs('./data/dgraph/train_feats.bin')
valid_feats, _ = load_graphs('./data/dgraph/valid_feats.bin')
test_feats, _ = load_graphs('./data/dgraph/test_feats.bin')

train_labels = torch.load("./data/dgraph/train_labels.pt")
valid_labels = torch.load("./data/dgraph/valid_labels.pt")
test_labels = torch.load("./data/dgraph/test_labels.pt")

In [4]:
train_feats[0], valid_feats[0]

(Graph(num_nodes={'A': 521, 'B': 916, 'C': 307},
       num_edges={('A', '10_t1', 'A'): 42, ('A', '10_t1', 'B'): 46, ('A', '10_t1', 'C'): 32, ('A', '11_t1', 'A'): 10, ('A', '11_t1', 'B'): 15, ('A', '11_t1', 'C'): 9, ('A', '9_t1', 'A'): 15, ('A', '9_t1', 'B'): 58, ('A', '9_t1', 'C'): 14, ('B', '10_t1', 'A'): 64, ('B', '10_t1', 'B'): 142, ('B', '10_t1', 'C'): 78, ('B', '11_t1', 'A'): 23, ('B', '11_t1', 'B'): 72, ('B', '11_t1', 'C'): 17, ('B', '9_t1', 'A'): 89, ('B', '9_t1', 'B'): 18, ('B', '9_t1', 'C'): 26, ('C', '10_t1', 'A'): 23, ('C', '10_t1', 'B'): 25, ('C', '10_t1', 'C'): 12, ('C', '11_t1', 'A'): 1, ('C', '11_t1', 'B'): 6, ('C', '11_t1', 'C'): 6, ('C', '9_t1', 'A'): 21, ('C', '9_t1', 'B'): 11, ('C', '9_t1', 'C'): 5},
       metagraph=[('A', 'A', '10_t1'), ('A', 'A', '11_t1'), ('A', 'A', '9_t1'), ('A', 'B', '10_t1'), ('A', 'B', '11_t1'), ('A', 'B', '9_t1'), ('A', 'C', '10_t1'), ('A', 'C', '11_t1'), ('A', 'C', '9_t1'), ('B', 'A', '10_t1'), ('B', 'A', '11_t1'), ('B', 'A', '9_t1'), ('B'

In [37]:
src_node, dst_node = train_feats[1][('A', '10_t1', 'B')].edges('uv')

src_node, dst_node + train_feats[1].nodes('A').shape[0]

(tensor([ 714,  617,  820, 1031,  711,  725,  369,  320,  597,  964,  381,  441,
          688,  636,  135,  190,  687,  468,  982, 1045,  612,  635,  558,  231,
          582,  279, 1021,  223,  581,  317,  563, 1025,  622,  580,  360,  158,
          915,  167,  777, 1040,  666,   91,  795,  835,  629,  912]),
 tensor([2035, 1423, 2451, 1839, 1161, 1667, 1601, 1883, 2281, 1639, 1582, 1140,
         1308, 1913, 2738, 2043, 1408, 2450, 1253, 1111, 1206, 1693, 2180, 2638,
         2712, 1386, 1330, 1283, 2453, 1842, 1631, 2014, 1244, 1766, 1375, 1338,
         2839, 1985, 2452, 1282, 1927, 1758, 2301, 2742, 1755, 2314]))

In [38]:
train_feats[1].canonical_etypes

[('A', '10_t1', 'A'),
 ('A', '10_t1', 'B'),
 ('A', '10_t1', 'C'),
 ('A', '10_t2', 'A'),
 ('A', '10_t2', 'B'),
 ('A', '10_t2', 'C'),
 ('A', '11_t1', 'A'),
 ('A', '11_t1', 'B'),
 ('A', '11_t1', 'C'),
 ('A', '11_t2', 'A'),
 ('A', '11_t2', 'B'),
 ('A', '11_t2', 'C'),
 ('A', '9_t1', 'A'),
 ('A', '9_t1', 'B'),
 ('A', '9_t1', 'C'),
 ('A', '9_t2', 'A'),
 ('A', '9_t2', 'B'),
 ('A', '9_t2', 'C'),
 ('B', '10_t1', 'A'),
 ('B', '10_t1', 'B'),
 ('B', '10_t1', 'C'),
 ('B', '10_t2', 'A'),
 ('B', '10_t2', 'B'),
 ('B', '10_t2', 'C'),
 ('B', '11_t1', 'A'),
 ('B', '11_t1', 'B'),
 ('B', '11_t1', 'C'),
 ('B', '11_t2', 'A'),
 ('B', '11_t2', 'B'),
 ('B', '11_t2', 'C'),
 ('B', '9_t1', 'A'),
 ('B', '9_t1', 'B'),
 ('B', '9_t1', 'C'),
 ('B', '9_t2', 'A'),
 ('B', '9_t2', 'B'),
 ('B', '9_t2', 'C'),
 ('C', '10_t1', 'A'),
 ('C', '10_t1', 'B'),
 ('C', '10_t1', 'C'),
 ('C', '10_t2', 'A'),
 ('C', '10_t2', 'B'),
 ('C', '10_t2', 'C'),
 ('C', '11_t1', 'A'),
 ('C', '11_t1', 'B'),
 ('C', '11_t1', 'C'),
 ('C', '11_t2', 'A'),


In [18]:
train_feats[1][('A', '10_t1', 'B')]

Graph(num_nodes={'A': 1074, 'B': 1824},
      num_edges={('A', '10_t1', 'B'): 46},
      metagraph=[('A', 'B', '10_t1')])

In [45]:
torch.cat(
    [
        train_feats[1].nodes['A'].data['feat'],
        train_feats[1].nodes['B'].data['feat']
    ],
    dim=0
)[train_feats[1].nodes('A').shape[0]:].shape

torch.Size([1824, 16])

In [67]:
_tmp, _ = torch.max(torch.stack([
    train_feats[1].nodes["B"].data["feat"].squeeze(),
    (train_feats[1].nodes["B"].data["feat"]*2).squeeze(),
], dim=1), dim=1)

_tmp.shape

torch.Size([1824, 16])

In [69]:
train_feats[1].nodes["B"].data["feat"]

tensor([[ 8.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 5.0000e+00,  7.3000e-01,  4.3500e-01,  ...,  4.0000e-01,
          6.8493e-03,  2.0000e-01],
        [ 5.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        ...,
        [ 8.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 4.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00]])

In [68]:
_tmp

tensor([[ 1.6000e+01, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 1.0000e+01,  1.4600e+00,  8.7000e-01,  ...,  8.0000e-01,
          1.3699e-02,  4.0000e-01],
        [ 1.0000e+01, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        ...,
        [ 1.6000e+01, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 8.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 4.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00]])

In [49]:
torch.cat(
    [
        train_feats[1].nodes['A'].data['feat'],
        train_feats[1].nodes['B'].data['feat']
    ],
    dim=0
).shape

torch.Size([2898, 16])

In [46]:
train_feats[1].nodes['B'].data['feat'].shape

torch.Size([1824, 16])